In [3]:
# import bus_processing
import bus_prediction
import os
import pandas as pd
from constants import DATA_DIR

/Users/legendword/python3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
def load_bus_data():
    path = os.path.join(DATA_DIR, "dataset.csv")
    df = pd.read_csv(path)
    return df

bus_data = load_bus_data()
bus_data["time_bucket"] = pd.to_datetime(bus_data["time_bucket"], utc=True)
X_train, y_train, X_test, y_test = bus_prediction.prepare_bus_data(bus_data)

## LSTM

In [6]:
lstm_model = bus_prediction.LSTMModule(input_size=X_train.shape[1]-2, hidden_size=64, num_layers=5, dropout=0.1)
lstm_predictor = bus_prediction.LSTMPredictor(lstm_model, window_size=10)
lstm_predictor.train(X_train, y_train, epochs=20)
mse = lstm_predictor.evaluate(X_test, y_test)
print(f"LSTM Test MSE: {mse:.4f}")

Epoch 1/20, Train Loss: 1.5367, Val Loss: 0.9385
Epoch 2/20, Train Loss: 1.4622, Val Loss: 0.9362
Epoch 3/20, Train Loss: 1.4603, Val Loss: 0.9350
Epoch 4/20, Train Loss: 1.4576, Val Loss: 0.9365
Epoch 5/20, Train Loss: 1.4496, Val Loss: 0.9036
Epoch 6/20, Train Loss: 1.4205, Val Loss: 0.9051
Epoch 7/20, Train Loss: 1.3635, Val Loss: 0.8516
Epoch 8/20, Train Loss: 1.3875, Val Loss: 0.8626
Epoch 9/20, Train Loss: 1.3312, Val Loss: 0.8015
Epoch 10/20, Train Loss: 1.3130, Val Loss: 0.7346
Epoch 11/20, Train Loss: 1.3264, Val Loss: 0.7460
Epoch 12/20, Train Loss: 1.3004, Val Loss: 0.6992
Epoch 13/20, Train Loss: 1.3218, Val Loss: 0.6931
Epoch 14/20, Train Loss: 1.2730, Val Loss: 0.6361
Epoch 15/20, Train Loss: 1.2559, Val Loss: 0.6170
Epoch 16/20, Train Loss: 1.2489, Val Loss: 0.6054
Epoch 17/20, Train Loss: 1.2934, Val Loss: 0.6360
Epoch 18/20, Train Loss: 1.2493, Val Loss: 0.6228
Epoch 19/20, Train Loss: 1.2184, Val Loss: 0.6276
Epoch 20/20, Train Loss: 1.2167, Val Loss: 0.5866


RuntimeError: input.size(-1) must be equal to input_size. Expected 14, got 15

In [ ]:
# LSTM previous output

Shape of y_pred: (2192,)
Shape of test_y: (2192,)
Test MSE: 1.0221


## XGBoost

In [3]:
xgboost_predictor = bus_prediction.XGBoostPredictor(lagged_features=["avg_delay"], lag_offsets=[1, 2, 3, 24*3], avg_features=["avg_delay"], avg_ranges=[24*3, 48*3, 72*3])
xgboost_predictor.train(X_train, y_train)
xgboost_mse = xgboost_predictor.evaluate(X_test, y_test)
print(f"XGBoost Test MSE: {xgboost_mse:.4f}")

Validation MSE: 0.1947
Shape of y_pred: (2130,)
Shape of test_y: (2130,)
XGBoost Test MSE: 0.3340


## TFT

In [3]:
tft_predictor = bus_prediction.TFTPredictor()
tft_predictor.train(X_train, y_train)

/Users/legendword/python3/lib/python3.12/site-packages/pytorch_forecasting/data/encoders.py:400: UserWarning: Found 1 unknown classes which were set to NaN
  warnings.warn(
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [5]:
tft_predictor.evaluate(X_test, y_test)

Restoring states from the checkpoint path at /Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/lightning_logs/version_80/checkpoints/epoch=1-step=344.ckpt
Loaded model weights from the checkpoint at /Users/legendword/Documents/zwh/Coding/CPSC440/cpsc440-transit-project/src/lightning_logs/version_80/checkpoints/epoch=1-step=344.ckpt
/Users/legendword/python3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_MAE          │    0.15751099586486816    │
│         test_MAPE         │    0.1002342700958252     │
│         test_RMSE         │    0.15751099586486816    │
│        test_SMAPE         │    0.1055227741599083     │
│         test_loss         │    0.15751099586486816    │
└───────────────────────────┴───────────────────────────┘

In [3]:
xgboost_predictor = bus_prediction.XGBoostPredictor()
xgboost_predictor.train(X_train, y_train)
xgboost_mse = xgboost_predictor.evaluate(X_test, y_test)
print(f"XGBoost Test MSE: {xgboost_mse:.4f}")

Validation MSE: 1.0641
Shape of y_pred: (2202,)
Shape of test_y: (2202,)
XGBoost Test MSE: 0.9830
